In [1]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)

In [2]:
df = pd.read_csv("/media/rghoshal/D4A6E10AA6E0EE44/UGP/datasets/data.csv",index_col = 0)
df["sla"].fillna(0,inplace = True)
df["sla"] = df["sla"].groupby(pd.DatetimeIndex(df['time']).month).transform(lambda x: (x - x.mean()) / x.std())

bhopal = df[(df['lat'] == 23.75) & (df['lon'] == 76.25 ) ]
bhopal["precip"] = bhopal["precip"] * pd.DatetimeIndex(bhopal['time']).daysinmonth
kanpur = df[(df['lat'] == 26.25) & (df['lon'] == 81.25 ) ]
kanpur["precip"] = kanpur["precip"] * pd.DatetimeIndex(kanpur['time']).daysinmonth
bombay = df[(df['lat'] == 18.75) & (df['lon'] == 71.25 ) ]
bombay["precip"] = bombay["precip"] * pd.DatetimeIndex(bombay['time']).daysinmonth

In [3]:
x = np.array(df["sla"]).reshape(312,20,26)
y = np.array(bhopal["precip"])
x_train = x[:240]
x_test = x[270:]
x_val = x[240:270]
bhopal_train = y[:240]
bhopal_test = y[270:]
bhopal_valid = y[240:270]
kanpur_train = y[:240]
kanpur_test = y[270:]
kanpur_valid = y[240:270]
bombay_train = y[:240]
bombay_test = y[270:]
bombay_valid = y[240:270]

In [4]:
x_train = x_train.reshape(x_train.shape[0], 20, 26, 1)
x_test = x_test.reshape(x_test.shape[0], 20, 26, 1)
x_val = x_val.reshape(x_val.shape[0], 20, 26, 1)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, LSTM, Embedding, TimeDistributed
from keras.metrics import RootMeanSquaredError as rmse

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(20,26,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(1,1), input_shape=(20,26,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Flatten())
model.add(TimeDistributed(Flatten())) # Flattening the 2D arrays for fully connected layers
# Add a LSTM layer with 128 internal units.
model.add(LSTM(128))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='relu'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 18, 24, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 12, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 12, 64)         2112      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 6, 64)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 384)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 512)             

In [8]:
model.compile(optimizer='adam', loss='mse', metrics=[rmse()])

In [9]:
model.fit(x=x_train,y=bhopal_train, epochs=500, batch_size = 64 , validation_data=(x_val, bhopal_valid))


Train on 240 samples, validate on 30 samples
Epoch 1/500
240/240 [==============================] - 1s 3ms/step - loss: 23639.0863 - root_mean_squared_error: 153.7501 - val_loss: 28501.3867 - val_root_mean_squared_error: 168.8235
Epoch 2/500
240/240 [==============================] - 0s 566us/step - loss: 23273.7482 - root_mean_squared_error: 152.5574 - val_loss: 27769.2598 - val_root_mean_squared_error: 166.6411
Epoch 3/500
240/240 [==============================] - 0s 546us/step - loss: 22571.2568 - root_mean_squared_error: 150.2373 - val_loss: 26650.9375 - val_root_mean_squared_error: 163.2511
Epoch 4/500
240/240 [==============================] - 0s 500us/step - loss: 21433.3033 - root_mean_squared_error: 146.4012 - val_loss: 25322.3066 - val_root_mean_squared_error: 159.1298
Epoch 5/500
240/240 [==============================] - 0s 498us/step - loss: 20275.3294 - root_mean_squared_error: 142.3915 - val_loss: 23962.3652 - val_root_mean_squared_error: 154.7978
Epoch 6/500
240/240 [

240/240 [==============================] - 0s 791us/step - loss: 3117.0996 - root_mean_squared_error: 55.8310 - val_loss: 7721.5166 - val_root_mean_squared_error: 87.8722
Epoch 44/500
240/240 [==============================] - 0s 1ms/step - loss: 2790.1011 - root_mean_squared_error: 52.8214 - val_loss: 7695.0728 - val_root_mean_squared_error: 87.7216
Epoch 45/500
240/240 [==============================] - 0s 723us/step - loss: 2754.4806 - root_mean_squared_error: 52.4831 - val_loss: 6566.0859 - val_root_mean_squared_error: 81.0314
Epoch 46/500
240/240 [==============================] - 0s 683us/step - loss: 2480.9589 - root_mean_squared_error: 49.8092 - val_loss: 7840.7769 - val_root_mean_squared_error: 88.5482
Epoch 47/500
240/240 [==============================] - 0s 487us/step - loss: 2386.9614 - root_mean_squared_error: 48.8565 - val_loss: 5924.4658 - val_root_mean_squared_error: 76.9706
Epoch 48/500
240/240 [==============================] - 0s 465us/step - loss: 2239.5013 - root_

Epoch 88/500
240/240 [==============================] - 0s 390us/step - loss: 694.9136 - root_mean_squared_error: 26.3612 - val_loss: 4733.5483 - val_root_mean_squared_error: 68.8008
Epoch 89/500
240/240 [==============================] - 0s 426us/step - loss: 707.3962 - root_mean_squared_error: 26.5969 - val_loss: 4565.5845 - val_root_mean_squared_error: 67.5691
Epoch 90/500
240/240 [==============================] - 0s 397us/step - loss: 644.5933 - root_mean_squared_error: 25.3888 - val_loss: 4638.1021 - val_root_mean_squared_error: 68.1036
Epoch 91/500
240/240 [==============================] - 0s 430us/step - loss: 588.9404 - root_mean_squared_error: 24.2681 - val_loss: 5080.4292 - val_root_mean_squared_error: 71.2771
Epoch 92/500
240/240 [==============================] - 0s 435us/step - loss: 578.3207 - root_mean_squared_error: 24.0483 - val_loss: 4584.9878 - val_root_mean_squared_error: 67.7125
Epoch 93/500
240/240 [==============================] - 0s 419us/step - loss: 543.134

240/240 [==============================] - 0s 471us/step - loss: 347.9512 - root_mean_squared_error: 18.6535 - val_loss: 4699.3652 - val_root_mean_squared_error: 68.5519
Epoch 133/500
240/240 [==============================] - 0s 520us/step - loss: 328.3306 - root_mean_squared_error: 18.1199 - val_loss: 4573.7827 - val_root_mean_squared_error: 67.6297
Epoch 134/500
240/240 [==============================] - 0s 663us/step - loss: 313.7291 - root_mean_squared_error: 17.7124 - val_loss: 4441.2920 - val_root_mean_squared_error: 66.6430
Epoch 135/500
240/240 [==============================] - 0s 465us/step - loss: 330.0227 - root_mean_squared_error: 18.1665 - val_loss: 4458.8511 - val_root_mean_squared_error: 66.7746
Epoch 136/500
240/240 [==============================] - 0s 474us/step - loss: 294.0288 - root_mean_squared_error: 17.1473 - val_loss: 4672.2642 - val_root_mean_squared_error: 68.3540
Epoch 137/500
240/240 [==============================] - 0s 503us/step - loss: 299.4913 - root

240/240 [==============================] - 0s 499us/step - loss: 291.3998 - root_mean_squared_error: 17.0704 - val_loss: 4505.2910 - val_root_mean_squared_error: 67.1215
Epoch 177/500
240/240 [==============================] - 0s 453us/step - loss: 282.5519 - root_mean_squared_error: 16.8093 - val_loss: 4466.4771 - val_root_mean_squared_error: 66.8317
Epoch 178/500
240/240 [==============================] - 0s 432us/step - loss: 284.3415 - root_mean_squared_error: 16.8624 - val_loss: 4564.8760 - val_root_mean_squared_error: 67.5639
Epoch 179/500
240/240 [==============================] - 0s 431us/step - loss: 286.9797 - root_mean_squared_error: 16.9405 - val_loss: 4686.9497 - val_root_mean_squared_error: 68.4613
Epoch 180/500
240/240 [==============================] - 0s 399us/step - loss: 272.0489 - root_mean_squared_error: 16.4939 - val_loss: 4812.1753 - val_root_mean_squared_error: 69.3698
Epoch 181/500
240/240 [==============================] - 0s 449us/step - loss: 278.4163 - root

...

Epoch 309/500
240/240 [==============================] - 0s 443us/step - loss: 67.3150 - root_mean_squared_error: 8.2046 - val_loss: 4642.3794 - val_root_mean_squared_error: 68.1350
Epoch 310/500
240/240 [==============================] - 0s 450us/step - loss: 90.6525 - root_mean_squared_error: 9.5212 - val_loss: 4774.6514 - val_root_mean_squared_error: 69.0989
Epoch 311/500
240/240 [==============================] - 0s 469us/step - loss: 58.3992 - root_mean_squared_error: 7.6419 - val_loss: 4743.8096 - val_root_mean_squared_error: 68.8753
Epoch 312/500
240/240 [==============================] - 0s 405us/step - loss: 61.0825 - root_mean_squared_error: 7.8155 - val_loss: 4739.5669 - val_root_mean_squared_error: 68.8445
Epoch 313/500
240/240 [==============================] - 0s 434us/step - loss: 59.1705 - root_mean_squared_error: 7.6922 - val_loss: 5024.0859 - val_root_mean_squared_error: 70.8808
Epoch 314/500
240/240 [==============================] - 0s 430us/step - loss: 47.5725 - r

240/240 [==============================] - 0s 550us/step - loss: 61.1223 - root_mean_squared_error: 7.8181 - val_loss: 5078.3940 - val_root_mean_squared_error: 71.2629
Epoch 355/500
240/240 [==============================] - 0s 498us/step - loss: 63.7628 - root_mean_squared_error: 7.9852 - val_loss: 5040.0562 - val_root_mean_squared_error: 70.9934
Epoch 356/500
240/240 [==============================] - 0s 498us/step - loss: 58.2687 - root_mean_squared_error: 7.6334 - val_loss: 4895.2495 - val_root_mean_squared_error: 69.9661
Epoch 357/500
240/240 [==============================] - 0s 513us/step - loss: 52.2054 - root_mean_squared_error: 7.2253 - val_loss: 4864.8545 - val_root_mean_squared_error: 69.7485
Epoch 358/500
240/240 [==============================] - 0s 536us/step - loss: 69.7071 - root_mean_squared_error: 8.3491 - val_loss: 4640.0762 - val_root_mean_squared_error: 68.1181
Epoch 359/500
240/240 [==============================] - 0s 559us/step - loss: 71.0846 - root_mean_squar

240/240 [==============================] - 0s 428us/step - loss: 99.7260 - root_mean_squared_error: 9.9863 - val_loss: 5077.8838 - val_root_mean_squared_error: 71.2593
Epoch 400/500
240/240 [==============================] - 0s 434us/step - loss: 84.8695 - root_mean_squared_error: 9.2125 - val_loss: 4637.6738 - val_root_mean_squared_error: 68.1005
Epoch 401/500
240/240 [==============================] - 0s 407us/step - loss: 79.1646 - root_mean_squared_error: 8.8975 - val_loss: 4471.5439 - val_root_mean_squared_error: 66.8696
Epoch 402/500
240/240 [==============================] - 0s 461us/step - loss: 57.1125 - root_mean_squared_error: 7.5573 - val_loss: 4741.2837 - val_root_mean_squared_error: 68.8570
Epoch 403/500
240/240 [==============================] - 0s 406us/step - loss: 69.4842 - root_mean_squared_error: 8.3357 - val_loss: 4681.4385 - val_root_mean_squared_error: 68.4210
Epoch 404/500
240/240 [==============================] - 0s 398us/step - loss: 78.5016 - root_mean_squar

240/240 [==============================] - 0s 538us/step - loss: 72.1271 - root_mean_squared_error: 8.4928 - val_loss: 4612.1577 - val_root_mean_squared_error: 67.9129
Epoch 445/500
240/240 [==============================] - 0s 530us/step - loss: 60.4575 - root_mean_squared_error: 7.7754 - val_loss: 4682.3657 - val_root_mean_squared_error: 68.4278
Epoch 446/500
240/240 [==============================] - 0s 481us/step - loss: 103.2666 - root_mean_squared_error: 10.1620 - val_loss: 4788.3022 - val_root_mean_squared_error: 69.1976
Epoch 447/500
240/240 [==============================] - 0s 494us/step - loss: 51.4331 - root_mean_squared_error: 7.1717 - val_loss: 4598.6021 - val_root_mean_squared_error: 67.8130
Epoch 448/500
240/240 [==============================] - 0s 503us/step - loss: 66.0912 - root_mean_squared_error: 8.1297 - val_loss: 4787.8770 - val_root_mean_squared_error: 69.1945
Epoch 449/500
240/240 [==============================] - 0s 494us/step - loss: 81.1014 - root_mean_squ

240/240 [==============================] - 0s 514us/step - loss: 51.2633 - root_mean_squared_error: 7.1598 - val_loss: 4933.3301 - val_root_mean_squared_error: 70.2377
Epoch 490/500
240/240 [==============================] - 0s 498us/step - loss: 53.1330 - root_mean_squared_error: 7.2892 - val_loss: 5161.9473 - val_root_mean_squared_error: 71.8467
Epoch 491/500
240/240 [==============================] - 0s 493us/step - loss: 84.7936 - root_mean_squared_error: 9.2083 - val_loss: 4890.6929 - val_root_mean_squared_error: 69.9335
Epoch 492/500
240/240 [==============================] - 0s 535us/step - loss: 60.5767 - root_mean_squared_error: 7.7831 - val_loss: 4719.0869 - val_root_mean_squared_error: 68.6956
Epoch 493/500
240/240 [==============================] - 0s 517us/step - loss: 62.5550 - root_mean_squared_error: 7.9092 - val_loss: 5295.0737 - val_root_mean_squared_error: 72.7673
Epoch 494/500
240/240 [==============================] - 0s 494us/step - loss: 88.4775 - root_mean_squar

In [ ]:
test_error_rate = model.evaluate(x_test, bhopal_test, verbose=0)
print("The mean squared error(MSE), Root mean squared error(RMSE) for the Bhopal set is:\n {}".format(test_error_rate))

In [32]:
model.fit(x=x_train,y=kanpur_train, epochs=500 , batch_size = 64 , validation_data=(x_val, kanpur_valid))

Train on 240 samples, validate on 30 samples
Epoch 1/500
240/240 [==============================] - 0s 485us/step - loss: 109.5913 - root_mean_squared_error: 10.4686 - val_loss: 4821.2061 - val_root_mean_squared_error: 69.4349
Epoch 2/500
240/240 [==============================] - 0s 483us/step - loss: 98.6721 - root_mean_squared_error: 9.9334 - val_loss: 4546.8892 - val_root_mean_squared_error: 67.4306
Epoch 3/500
240/240 [==============================] - 0s 471us/step - loss: 72.1645 - root_mean_squared_error: 8.4950 - val_loss: 4989.0322 - val_root_mean_squared_error: 70.6331
Epoch 4/500
240/240 [==============================] - 0s 503us/step - loss: 67.7615 - root_mean_squared_error: 8.2317 - val_loss: 5213.3911 - val_root_mean_squared_error: 72.2038
Epoch 5/500
240/240 [==============================] - 0s 480us/step - loss: 52.5861 - root_mean_squared_error: 7.2516 - val_loss: 5468.2974 - val_root_mean_squared_error: 73.9479
Epoch 6/500
240/240 [==============================] 

240/240 [==============================] - 0s 450us/step - loss: 23.3198 - root_mean_squared_error: 4.8291 - val_loss: 5300.6963 - val_root_mean_squared_error: 72.8059
Epoch 47/500
240/240 [==============================] - 0s 408us/step - loss: 33.2125 - root_mean_squared_error: 5.7630 - val_loss: 5133.3677 - val_root_mean_squared_error: 71.6475
Epoch 48/500
240/240 [==============================] - 0s 451us/step - loss: 30.7170 - root_mean_squared_error: 5.5423 - val_loss: 5335.1382 - val_root_mean_squared_error: 73.0420
Epoch 49/500
240/240 [==============================] - 0s 447us/step - loss: 45.9437 - root_mean_squared_error: 6.7782 - val_loss: 5172.7817 - val_root_mean_squared_error: 71.9221
Epoch 50/500
240/240 [==============================] - 0s 441us/step - loss: 43.8664 - root_mean_squared_error: 6.6232 - val_loss: 5299.4785 - val_root_mean_squared_error: 72.7975
Epoch 51/500
240/240 [==============================] - 0s 457us/step - loss: 40.1818 - root_mean_squared_er

240/240 [==============================] - 0s 422us/step - loss: 42.3576 - root_mean_squared_error: 6.5083 - val_loss: 5219.6377 - val_root_mean_squared_error: 72.2471
Epoch 92/500
240/240 [==============================] - 0s 661us/step - loss: 21.6941 - root_mean_squared_error: 4.6577 - val_loss: 5151.3979 - val_root_mean_squared_error: 71.7732
Epoch 93/500
240/240 [==============================] - 0s 384us/step - loss: 20.4724 - root_mean_squared_error: 4.5246 - val_loss: 5220.5454 - val_root_mean_squared_error: 72.2533
Epoch 94/500
240/240 [==============================] - 0s 382us/step - loss: 23.6898 - root_mean_squared_error: 4.8672 - val_loss: 5250.8765 - val_root_mean_squared_error: 72.4629
Epoch 95/500
240/240 [==============================] - 0s 528us/step - loss: 27.2897 - root_mean_squared_error: 5.2240 - val_loss: 5037.1685 - val_root_mean_squared_error: 70.9730
Epoch 96/500
240/240 [==============================] - 0s 417us/step - loss: 22.3714 - root_mean_squared_er

240/240 [==============================] - 0s 525us/step - loss: 31.3818 - root_mean_squared_error: 5.6019 - val_loss: 5299.2476 - val_root_mean_squared_error: 72.7959
Epoch 137/500
240/240 [==============================] - 0s 436us/step - loss: 32.9281 - root_mean_squared_error: 5.7383 - val_loss: 4959.3706 - val_root_mean_squared_error: 70.4228
Epoch 138/500
240/240 [==============================] - 0s 425us/step - loss: 32.3522 - root_mean_squared_error: 5.6879 - val_loss: 5183.3662 - val_root_mean_squared_error: 71.9956
Epoch 139/500
240/240 [==============================] - 0s 550us/step - loss: 27.1440 - root_mean_squared_error: 5.2100 - val_loss: 5219.8018 - val_root_mean_squared_error: 72.2482
Epoch 140/500
240/240 [==============================] - 0s 530us/step - loss: 26.2978 - root_mean_squared_error: 5.1281 - val_loss: 4938.3271 - val_root_mean_squared_error: 70.2732
Epoch 141/500
240/240 [==============================] - 0s 401us/step - loss: 30.7623 - root_mean_squar

240/240 [==============================] - 0s 324us/step - loss: 28.6622 - root_mean_squared_error: 5.3537 - val_loss: 4801.3481 - val_root_mean_squared_error: 69.2918
Epoch 182/500
240/240 [==============================] - 0s 325us/step - loss: 29.0704 - root_mean_squared_error: 5.3917 - val_loss: 4869.6411 - val_root_mean_squared_error: 69.7828
Epoch 183/500
240/240 [==============================] - 0s 323us/step - loss: 31.8921 - root_mean_squared_error: 5.6473 - val_loss: 4890.9673 - val_root_mean_squared_error: 69.9354
Epoch 184/500
240/240 [==============================] - 0s 318us/step - loss: 24.3446 - root_mean_squared_error: 4.9340 - val_loss: 4893.6187 - val_root_mean_squared_error: 69.9544
Epoch 185/500
240/240 [==============================] - 0s 325us/step - loss: 32.1464 - root_mean_squared_error: 5.6698 - val_loss: 5257.5942 - val_root_mean_squared_error: 72.5093
Epoch 186/500
240/240 [==============================] - 0s 322us/step - loss: 28.2731 - root_mean_squar

...

240/240 [==============================] - 0s 356us/step - loss: 32.3579 - root_mean_squared_error: 5.6884 - val_loss: 5213.9717 - val_root_mean_squared_error: 72.2078
Epoch 317/500
240/240 [==============================] - 0s 359us/step - loss: 17.9515 - root_mean_squared_error: 4.2369 - val_loss: 5086.4722 - val_root_mean_squared_error: 71.3195
Epoch 318/500
240/240 [==============================] - 0s 382us/step - loss: 30.4143 - root_mean_squared_error: 5.5149 - val_loss: 5220.0635 - val_root_mean_squared_error: 72.2500
Epoch 319/500
240/240 [==============================] - 0s 446us/step - loss: 25.0782 - root_mean_squared_error: 5.0078 - val_loss: 5422.0391 - val_root_mean_squared_error: 73.6345
Epoch 320/500
240/240 [==============================] - 0s 334us/step - loss: 29.2745 - root_mean_squared_error: 5.4106 - val_loss: 5428.3350 - val_root_mean_squared_error: 73.6772
Epoch 321/500
240/240 [==============================] - 0s 415us/step - loss: 27.0426 - root_mean_squar

240/240 [==============================] - 0s 357us/step - loss: 25.2775 - root_mean_squared_error: 5.0277 - val_loss: 5278.6191 - val_root_mean_squared_error: 72.6541
Epoch 362/500
240/240 [==============================] - 0s 319us/step - loss: 25.7252 - root_mean_squared_error: 5.0720 - val_loss: 5297.2246 - val_root_mean_squared_error: 72.7820
Epoch 363/500
240/240 [==============================] - 0s 334us/step - loss: 28.5593 - root_mean_squared_error: 5.3441 - val_loss: 5189.6333 - val_root_mean_squared_error: 72.0391
Epoch 364/500
240/240 [==============================] - 0s 329us/step - loss: 25.5076 - root_mean_squared_error: 5.0505 - val_loss: 5156.9268 - val_root_mean_squared_error: 71.8117
Epoch 365/500
240/240 [==============================] - 0s 322us/step - loss: 31.8368 - root_mean_squared_error: 5.6424 - val_loss: 4833.6616 - val_root_mean_squared_error: 69.5245
Epoch 366/500
240/240 [==============================] - 0s 319us/step - loss: 32.0323 - root_mean_squar

240/240 [==============================] - 0s 323us/step - loss: 23.7063 - root_mean_squared_error: 4.8689 - val_loss: 4969.5854 - val_root_mean_squared_error: 70.4953
Epoch 407/500
240/240 [==============================] - 0s 320us/step - loss: 29.1412 - root_mean_squared_error: 5.3983 - val_loss: 4866.7393 - val_root_mean_squared_error: 69.7620
Epoch 408/500
240/240 [==============================] - 0s 320us/step - loss: 30.5907 - root_mean_squared_error: 5.5309 - val_loss: 4898.3198 - val_root_mean_squared_error: 69.9880
Epoch 409/500
240/240 [==============================] - 0s 316us/step - loss: 27.1951 - root_mean_squared_error: 5.2149 - val_loss: 5067.5186 - val_root_mean_squared_error: 71.1865
Epoch 410/500
240/240 [==============================] - 0s 321us/step - loss: 34.4259 - root_mean_squared_error: 5.8674 - val_loss: 4843.8408 - val_root_mean_squared_error: 69.5977
Epoch 411/500
240/240 [==============================] - 0s 358us/step - loss: 32.3769 - root_mean_squar

240/240 [==============================] - 0s 333us/step - loss: 24.5126 - root_mean_squared_error: 4.9510 - val_loss: 4901.7603 - val_root_mean_squared_error: 70.0126
Epoch 452/500
240/240 [==============================] - 0s 329us/step - loss: 25.2751 - root_mean_squared_error: 5.0274 - val_loss: 4948.0200 - val_root_mean_squared_error: 70.3422
Epoch 453/500
240/240 [==============================] - 0s 330us/step - loss: 27.1035 - root_mean_squared_error: 5.2061 - val_loss: 4859.7725 - val_root_mean_squared_error: 69.7121
Epoch 454/500
240/240 [==============================] - 0s 373us/step - loss: 35.8592 - root_mean_squared_error: 5.9883 - val_loss: 4909.8960 - val_root_mean_squared_error: 70.0706
Epoch 455/500
240/240 [==============================] - 0s 343us/step - loss: 18.9718 - root_mean_squared_error: 4.3557 - val_loss: 5199.3213 - val_root_mean_squared_error: 72.1063
Epoch 456/500
240/240 [==============================] - 0s 321us/step - loss: 26.6014 - root_mean_squar

240/240 [==============================] - 0s 341us/step - loss: 28.9690 - root_mean_squared_error: 5.3823 - val_loss: 4955.4116 - val_root_mean_squared_error: 70.3947
Epoch 497/500
240/240 [==============================] - 0s 335us/step - loss: 26.7043 - root_mean_squared_error: 5.1676 - val_loss: 4564.7466 - val_root_mean_squared_error: 67.5629
Epoch 498/500
240/240 [==============================] - 0s 415us/step - loss: 25.1161 - root_mean_squared_error: 5.0116 - val_loss: 4901.0259 - val_root_mean_squared_error: 70.0073
Epoch 499/500
240/240 [==============================] - 0s 519us/step - loss: 24.7970 - root_mean_squared_error: 4.9797 - val_loss: 5277.2012 - val_root_mean_squared_error: 72.6443
Epoch 500/500
240/240 [==============================] - 0s 325us/step - loss: 23.5424 - root_mean_squared_error: 4.8520 - val_loss: 4922.4941 - val_root_mean_squared_error: 70.1605


In [53]:
test_error_rate = model.evaluate(x_test, kanpur_test, verbose=0)
print("The mean squared error(MSE), Root mean squared error(RMSE) for the Kanpur set is:\n {}".format(test_error_rate))

The mean squared error(MSE), Root mean squared error(RMSE) for the Kanpur set is:
 [4397.353786892361, 66.31254577636719]


In [13]:
model.fit(x=x_train,y=bombay_train, epochs=500 , batch_size = 64 , validation_data=(x_val, bombay_valid))

Epoch 1/500
252/252 [==============================] - 0s 980us/step - loss: 0.0279 - root_mean_squared_error: 0.1670
Epoch 2/500
252/252 [==============================] - 0s 696us/step - loss: 0.0571 - root_mean_squared_error: 0.2391
Epoch 3/500
252/252 [==============================] - 0s 881us/step - loss: 0.0225 - root_mean_squared_error: 0.1500
Epoch 4/500
252/252 [==============================] - 0s 706us/step - loss: 0.0327 - root_mean_squared_error: 0.1809
Epoch 5/500
252/252 [==============================] - 0s 389us/step - loss: 0.0457 - root_mean_squared_error: 0.2137
Epoch 6/500
252/252 [==============================] - 0s 467us/step - loss: 0.0601 - root_mean_squared_error: 0.2451
Epoch 7/500
252/252 [==============================] - 0s 640us/step - loss: 0.0314 - root_mean_squared_error: 0.1773
Epoch 8/500
252/252 [==============================] - 0s 624us/step - loss: 0.0577 - root_mean_squared_error: 0.2401
Epoch 9/500
252/252 [==============================] - 0

252/252 [==============================] - 0s 408us/step - loss: 0.0314 - root_mean_squared_error: 0.1773
Epoch 138/500
252/252 [==============================] - 0s 392us/step - loss: 0.0226 - root_mean_squared_error: 0.1504
Epoch 139/500
252/252 [==============================] - 0s 460us/step - loss: 0.0317 - root_mean_squared_error: 0.1780
Epoch 140/500
252/252 [==============================] - 0s 456us/step - loss: 0.0220 - root_mean_squared_error: 0.1484
Epoch 141/500
252/252 [==============================] - 0s 584us/step - loss: 0.0322 - root_mean_squared_error: 0.1794
Epoch 142/500
252/252 [==============================] - 0s 429us/step - loss: 0.0415 - root_mean_squared_error: 0.2037
Epoch 143/500
252/252 [==============================] - 0s 381us/step - loss: 0.0441 - root_mean_squared_error: 0.2099
Epoch 144/500
252/252 [==============================] - 0s 412us/step - loss: 0.0333 - root_mean_squared_error: 0.1824
Epoch 145/500
252/252 [==============================]

252/252 [==============================] - 0s 406us/step - loss: 0.0309 - root_mean_squared_error: 0.1757
Epoch 274/500
252/252 [==============================] - 0s 396us/step - loss: 0.0277 - root_mean_squared_error: 0.1665
Epoch 275/500
252/252 [==============================] - 0s 395us/step - loss: 0.0233 - root_mean_squared_error: 0.1526
Epoch 276/500
252/252 [==============================] - 0s 382us/step - loss: 0.0277 - root_mean_squared_error: 0.1664
Epoch 277/500
252/252 [==============================] - 0s 395us/step - loss: 0.0275 - root_mean_squared_error: 0.1659
Epoch 278/500
252/252 [==============================] - 0s 381us/step - loss: 0.0366 - root_mean_squared_error: 0.1913
Epoch 279/500
252/252 [==============================] - 0s 609us/step - loss: 0.0608 - root_mean_squared_error: 0.2466
Epoch 280/500
252/252 [==============================] - 0s 548us/step - loss: 0.1052 - root_mean_squared_error: 0.3244
Epoch 281/500
252/252 [==============================]

252/252 [==============================] - 0s 478us/step - loss: 0.0274 - root_mean_squared_error: 0.1656
Epoch 410/500
252/252 [==============================] - 0s 457us/step - loss: 0.0294 - root_mean_squared_error: 0.1713
Epoch 411/500
252/252 [==============================] - 0s 443us/step - loss: 0.0266 - root_mean_squared_error: 0.1631
Epoch 412/500
252/252 [==============================] - 0s 940us/step - loss: 0.0339 - root_mean_squared_error: 0.1842
Epoch 413/500
252/252 [==============================] - 0s 566us/step - loss: 0.0209 - root_mean_squared_error: 0.1444
Epoch 414/500
252/252 [==============================] - 0s 497us/step - loss: 0.0257 - root_mean_squared_error: 0.1604
Epoch 415/500
252/252 [==============================] - 0s 473us/step - loss: 0.0434 - root_mean_squared_error: 0.2083
Epoch 416/500
252/252 [==============================] - 0s 481us/step - loss: 0.0295 - root_mean_squared_error: 0.1716
Epoch 417/500
252/252 [==============================]

In [14]:
test_error_rate = model.evaluate(x_test, bombay_test, verbose=0)
print("The mean squared error(MSE), Root mean squared error(RMSE) for the Bombay set is:\n {}".format(test_error_rate))

The mean squared error(MSE), Root mean squared error(RMSE) for the Bombay set is:
 [5.520784091949463, 2.349634885787964]
